Goal: 
    - Extract unique names from subject and title and search in wikipedia information about them
    
Next Steps:
    - Improve the search using year (extract year from records and match with wikipedia description date) for the items with more than one reference attached in wikipedia

In [1]:
import json
import re
import collections
import numpy as np
import multiprocessing

import wikipedia
import pandas as pd

wikipedia.set_lang('en')

from fdh_gallica import Search

N_PROC = 8

/Users/lia/miniconda3/lib/python3.6/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
# search = Search(all_fields="atelier nadar", dc_type="image", dc_creator='atelier nadar')
# search.execute()
# search.total_records
# with open('raw_records.json', 'w') as fp:
#     json.dump(search.records, fp)
with open('raw_records.json', 'r') as fp:
    raw_records = json.load(fp)

In [3]:
##### Copied from Cleaning data notebook #####
def find_names(name):
    return re.findall('([\S]* [\S]*|[\S]*|[\S]*\, [\S]*[ \S*]+) \([\d]{2}', name)

def find_names_undated(name):
    # mo
    return re.findall('([\S]*|[\S]*\, [\S]*[ \S*]+) -- Portraits', name)

def find_names_title(name):
    return [x.strip() for x in re.findall('([\S\s ^(\:)]*) \: ', name) if len(x.strip()) > 0]

def extract_from_brackets(name):
    return re.findall('\[([\w \-\,\.]*)\]', name)

def extract_caricature(name):
    return name.split('Caricature de')[-1].split('pour')[0][:-2]

In [17]:
def get_names(data):
    
    # in subject
    subject_lists = [[r['dc:subject']] if isinstance(r['dc:subject'], str) 
                     else r['dc:subject'] 
                     for r in raw_records]
    
    names_subject_with_date = [find_names(item) 
                               for sub in subject_lists
                               for item in sub if find_names(item)]  
    
    flat_names_subject_with_date = [item 
                                    for sub in names_subject_with_date 
                                    for item in sub]
    
    names_subject_without_date = [find_names_undated(item) 
                                  for sub in subject_lists
                                  for item in sub if find_names_undated(item)]
    flat_names_subject_without_date = [item.split('(')[0][:-2]
                                            for sub in names_subject_without_date
                                            for item in sub]
    
    # in title
    title_lists = [[r['dc:title']] if isinstance(r['dc:title'], str) 
                     else r['dc:title'] 
                     for r in raw_records]
    
    names_title = [find_names_title(item) 
                   for sub in title_lists
                   for item in sub if find_names_title(item)] 
    
    flat_names_title = [item.replace('[', '').replace(']', '')
                        for sub in names_title
                        for item in sub]
    
    all_names = flat_names_subject_with_date + flat_names_subject_without_date + flat_names_title
    
    all_names = [extract_caricature(x) for x in all_names]
    
    unique_names = [x for x in list(set(all_names)) if x!='']
    
    return unique_names

In [22]:
all_names = get_names(raw_records)
print(len(all_names))
all_names[987:1000]

12644


['M. Fugère et Landr',
 'Savenay. Variétés. La rev',
 'Onésime Recl',
 'Couralet. Folies Bergère. "Les Poules de Grévi',
 'M. Selli',
 'Arnaud, Alb',
 'Mlle Guitty. Gaîté. Bicyclistes en voya',
 'Coche',
 'Tardieu, Ambroi',
 'Baugniet, Charl',
 'Me Carnot. Mère du président de la Républiq',
 'Gildès, Anth',
 'Chavannes, Pier']

In [23]:
wikipedia.search("Plantade, Charles-François (1787-1870)")

['Charles-François Plantade', '1787 in music']

In [47]:
def hacked_search(x):
    return wikipedia.search(x) # just pure functions can be pickled and distributed

pool = multiprocessing.Pool(processes=N_PROC)
wiki_searchs = dict(zip(all_names, pool.map(hacked_search, all_names)))
len(wiki_searchs)

12644

/Users/lia/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/lia/miniconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/Users/lia/miniconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code 

In [48]:
collections.Counter([len(x) for x in wiki_searchs.values()])

Counter({0: 7525,
         2: 359,
         1: 648,
         10: 3236,
         3: 225,
         6: 95,
         5: 145,
         7: 84,
         4: 182,
         8: 97,
         9: 48})

In [49]:
with_0_wiki = [k for k,v in wiki_searchs.items() if len(v)==0]
with_0_wiki[3:6]

['Saint-Georges, Alexandre-Pierre-Thomas-Amab',
 'Weill. Folies Dramatiques. La fauvet',
 'Sanlaville. Odéon. Le mariage de Figa']

In [52]:
def hacked_summary(x):
    try:
        s = wikipedia.summary(x, sentences=2)
        return s
    exp

wiki_summaries = dict(zip(all_names, pool.map(hacked_summary, all_names)))
len(wiki_summaries)

PageError: Page id "gilbert lydia" does not match any pages. Try another id!

In [51]:
wiki_summaries

{'Félix Nadar, écrivant à son bure': [],
 'Lucie. Folies-Bergè': ['Multiple-language version',
  'Josephine, Guardian Angel'],
 'M. Samuel (Variétés). Directe': ['Timeline of Paris'],
 'Mme Thévin et Capoul. (Lyrique). Paul et Virgin': [],
 'Sisos, Raphaë': [],
 'Decori, Lo': ['Royal Institute of British Architects'],
 'Me Ser': ['CSS (band)',
  'Mina El Hammani',
  'Edmund Ser',
  'Soy Como Quiero Ser',
  'Ernest Ansermet',
  'Tales of Dunk and Egg',
  'Leonardo da Vinci',
  'Catalytic triad',
  'Personal life of Leonardo da Vinci',
  'El Alma al Aire'],
 'Trivi': ['Rudi Stern',
  'Speech synthesis',
  'Roman Traditional Movement',
  'List of provincial and territorial nicknames in Canada',
  'Latin conjugation',
  'Saint Vincent and the Grenadines national football team',
  '2018 CONCACAF U-20 Championship squads',
  'Arturo Reghini',
  '2018 CONCACAF U-20 Championship',
  'List of inscriptions in Serbia'],
 'Saint-Georges, Alexandre-Pierre-Thomas-Amab': [],
 'Martin et Dupont-Vernon

In [53]:
np.min([12,3,4])

3

In [55]:
[1,2,3].min

AttributeError: 'list' object has no attribute 'min'

In [56]:
range(i+1, len(l))

range(0, 10)